In [18]:
%load_ext autoreload
%autoreload 2

In [19]:
from pathlib import Path
import pandas as pd
import numpy as np
import itertools
from copy import copy
from pprint import pprint
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import abc
import sys
import re
sys.path.append("../")
import gpusims
from gpusims.config import Config, parse_configs
from gpusims.bench import parse_benchmarks

In [20]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [21]:
root_dir = Path("/Users/roman/dev/gpgpusims")
benchmark_dir = root_dir / "benchmarks"
run_dir = root_dir / "run"
assert benchmark_dir.is_dir()
assert run_dir.is_dir()

In [22]:
configs = parse_configs(benchmark_dir / "configs" / "configs.yml")
benchmarks = parse_benchmarks(benchmark_dir / "benchmarks.yml")

pprint(configs)
pprint(benchmarks)
pprint(gpusims.SIMULATORS)

{'sm6_gtx1080': Config(name='SM6_GTX1080', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM6_GTX1080'), spec={'sm_count': 28, 'clock_speed': 1481}),
 'sm86_rtx3070': Config(name='SM86_RTX3070', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM86_RTX3070'), spec={'sm_count': 46, 'clock_speed': 1132})}
{'babelstream': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/BabelStream),
 'cuda10-matrixmul': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-matrixMul),
 'cuda10-transpose': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-transpose),
 'cuda4-matrixmul': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA4-matrixMul),
 'cuda6-transpose': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA6-transpose),
 'vectoradd': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/vectorAdd)}
{'accelsim-ptx': <class 'gpusims.accelsim.AccelSimPTXBenchmarkConfig'>,
 'accelsim-sass': <class 'gpusims.accelsim_sass.AccelSimSASSBenchmarkConfig'>,
 'm2s': <class '

In [23]:
selected_simulators = copy(gpusims.SIMULATORS)
selected_configs = copy(configs)
selected_benchmarks = copy(benchmarks)

In [24]:
class PlotData:
    def __init__(
        self,
        hw_df=None,
        m2s_df=None,
        macsim_df=None,
        tejas_df=None,
        accel_ptx_df=None,
        accel_sass_df=None,
    ):
        self.config = config
        self.hw_df = hw_df
        self.m2s_df = m2s_df
        self.macsim_df = macsim_df
        self.tejas_df = tejas_df
        self.accel_ptx_df = accel_ptx_df
        self.accel_sass_df = accel_sass_df
        
    def __getitem__(self, key):
        return getattr(self, key)

    def __setitem__(self, key, value):
        setattr(self, key, value)

In [25]:
colors= ['#0F8C79','#BD2D28','#E3BA22','#E6842A','#137B80','#8E6C8A','#9A3E25', '#E6842A']
colors= ["#5c0048", "#584588", "#3e78b7", "#39a8d1", "#70d5dd", "#b9ffe9"]
sim_color = {
    "GPUTejas": colors[0],
    "AccelSim PTX": colors[1],
    "AccelSim SASS": colors[2],
    "Hardware": colors[3],
    "Multi2Sim": colors[4],
    "MacSim": colors[5],
}

In [26]:
def plot_bars(config, bench, inp, data):
    metrics = {
        "Cycles": gpusims.plot.metrics.Cycles(data),
        "L2ReadHit": gpusims.plot.metrics.L2ReadHit(data),
        "DRAMReads": gpusims.plot.metrics.DRAMReads(data),
        "DRAMWrites": gpusims.plot.metrics.DRAMWrites(data),
        "IPC": gpusims.plot.metrics.IPC(data),
        "InstructionCount": gpusims.plot.metrics.InstructionCount(data),
    }
    for metric_name, metric in metrics.items():
        print("######")
        print(metric_name)
        print(metric.compute())

In [27]:
all_plot_data = []

# per config, benchmark and input, plot bars for each simulator
for (config_name, config), (bench_name, bench) in itertools.product(
    selected_configs.items(),
    selected_benchmarks.items()
):
    # if gpusims.NATIVE in selected_simulators and bench.enabled(gpusims.NATIVE):
    for inp in bench.inputs:
        plot_data = PlotData()
        for (sim_name, sim) in selected_simulators.items():
            if not bench.enabled(sim_name):
                continue
            bench_config = sim(
                run_dir=run_dir / sim_name.lower(),
                benchmark=bench,
                config=config,
            )
            plot_data[sim_name] = bench_config.load_dataframe(inp)
            print(sim_name, plot_data[sim_name] is None)
            # plot_dasim_nameta[sim_name] = 
            # print(bench_config)
            # for inp in benchmark.inputs:
            #     fabs = bench_config.load_dataframe(inp)
            # break
        # all_plot_data.append((config, bench, inp, plot_data))
        plot_bars(config, bench, inp, plot_data)
        break
    break

plot_data

kernels shape (1, 11)
cycles shape (1, 592)
inner join shape (1, 603)
native False
accelsim-ptx False
accelsim-sass False
tejas False
macsim False


AttributeError: module 'gpusims' has no attribute 'plot'

In [15]:
all_bench_configs = []

for (sim_name, sim), (config_name, config), (bench_name, bench) in itertools.product(
    selected_simulators.items(),
    selected_configs.items(),
    selected_benchmarks.items()
):
    if not bench.enabled(sim):
        continue
    
    # for inp in bench:
    #all_plot_configs.append(BenchmarkPlot(bench_config=sim(
    #    run_dir=run_dir / sim_name.lower(),
    #    benchmark=bench,
    #    config=config,
    #)))

print(f"{len(all_bench_configs)} total benchmark configs")

0 total benchmark configs


In [14]:
for plot_config in all_bench_configs:
    for inp in plot_config.bench_config.benchmark.inputs:
        hw_df = plot.load_hardware_df(inp)
        hw_df = plot.load_hardware_df(inp)
        accel_sass_df = pd.read_csv(accel_sass_results / "results/stats.csv")
print("accel sass shape", accel_sass_df.shape)
accel_sass_df = accel_sass_df.pivot(index=["kernel", "kernel_id"], columns=["stat"])["value"]
print("accel sass shape", accel_sass_df.shape)
# pprint(accel_df.columns.tolist())
accel_sass_df.T
        break
    break
hw_df.T
# plot = all_bench_configs[0]

kernels shape (1, 11)
cycles shape (1, 592)
inner join shape (1, 603)


Stream,7.0
Context,1.0
Device,NVIDIA GeForce GTX 1080 (0)
Kernel,_Z6vecAddPdS_S_i
Correlation_ID,112.0
Block X,1024.0
Block Y,1.0
Block Z,1.0
Duration,3.626667
Dynamic SMem,0.0
...,...


In [11]:
plot = all_bench_configs[0]
hw_df = plot.load_hardware_df()

NameError: name 'pd' is not defined

In [ ]:
hw_df = build_hw_df(
    cycle_csv_files=list((native_results / "results").rglob(r"result.cycles.csv.*")),
    kernel_csv_files=list((native_results / "results").rglob(r"result.csv.*")),
)
hw_df.T